## Import Data and necessary files

## Perform EDA and clean data

## Think of how you are going to cluster customer groups from the dataset that you have

### Types of customers => what features of the dataset you may need, IF OR NOT you may need to create new features